<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"


# Module 5: Pipeline and Grid Search

## Predicting Grant Applications: Cross Validation and Model Tuning

### Lesson Objectives

* After completing this lesson, you should be able to:
  - Avoid overfitting by using cross validation when training a model
  - Improve a model's performance by using grid search to find better parameters
  
### Choosing Parameters for Tuning

load grant data 

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._
import org.apache.spark.sql.functions._

val data = spark.read.
  format("com.databricks.spark.csv").
  option("delimiter", "\t").
  option("header", "true").
  option("inferSchema", "true").
  load("/resources/data/grantsPeople.csv")

data.show()

+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+----+----+------------+------------+--------------------+-------------------+
|Grant_Application_ID| RFCD_Code|RFCD_Percentage| SEO_Code|SEO_Percentage|Person_ID|                Role|Year_of_Birth|Country_of_Birth|Home_Language| Dept_No|Faculty_No|With_PHD|No_of_Years_in_Uni_at_Time_of_Grant|Number_of_Successful_Grant|Number_of_Unsuccessful_Grant|  A2|   A|   B|   C|Grant_Status|Sponsor_Code| Contract_Value_Band|Grant_Category_Code|
+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+-

spark = org.apache.spark.sql.SparkSession@460299b
data = [Grant_Application_ID: int, RFCD_Code: string ... 22 more fields]


[Grant_Application_ID: int, RFCD_Code: string ... 22 more fields]

create features 

In [2]:
val researchers = data.
  withColumn ("phd", data("With_PHD").equalTo("Yes").cast("Int")).
  withColumn ("CI", data("Role").equalTo("CHIEF_INVESTIGATOR").cast("Int")).
  withColumn("paperscore", data("A2") * 4 + data("A") * 3)

val grants = researchers.groupBy("Grant_Application_ID").agg(
  max("Grant_Status").as("Grant_Status"),
  max("Grant_Category_Code").as("Category_Code"),
  max("Contract_Value_Band").as("Value_Band"),
  sum("phd").as("PHDs"),
  when(max(expr("paperscore * CI")).isNull, 0).
    otherwise(max(expr("paperscore * CI"))).as("paperscore"),
  count("*").as("teamsize"),
  when(sum("Number_of_Successful_Grant").isNull, 0).
    otherwise(sum("Number_of_Successful_Grant")).as("successes"),
  when(sum("Number_of_Unsuccessful_Grant").isNull, 0).
    otherwise(sum("Number_of_Unsuccessful_Grant")).as("failures")
)

grants.show()

+--------------------+------------+-------------+--------------------+----+----------+--------+---------+--------+
|Grant_Application_ID|Grant_Status|Category_Code|          Value_Band|PHDs|paperscore|teamsize|successes|failures|
+--------------------+------------+-------------+--------------------+----+----------+--------+---------+--------+
|                 148|           0|  GrantCat30B|ContractValueBandUnk|null|         6|       1|        0|       1|
|                 463|           1|  GrantCat30C|ContractValueBandUnk|null|         0|       1|        1|       0|
|                 471|           0|  GrantCat30B|  ContractValueBandA|   1|       127|       2|        1|       5|
|                 496|           0|  GrantCat30B|  ContractValueBandA|null|         0|       1|        1|       3|
|                 833|           1|  GrantCat10A|  ContractValueBandF|null|         0|       1|        0|       0|
|                1088|           1|  GrantCat50A|  ContractValueBandA|   1|     

researchers = [Grant_Application_ID: int, RFCD_Code: string ... 25 more fields]
grants = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]


[Grant_Application_ID: int, Grant_Status: int ... 7 more fields]

convert string features to numbers 

In [3]:
import org.apache.spark.ml.feature.StringIndexer

val value_band_indexer = new StringIndexer().
  setInputCol("Value_Band").
  setOutputCol("Value_index").
  fit(grants)
  
val category_indexer = new StringIndexer().
  setInputCol("Category_Code").
  setOutputCol("Category_index").
  fit(grants)
  
val label_indexer = new StringIndexer().
  setInputCol("Grant_Status").
  setOutputCol("status").
  fit(grants)

value_band_indexer = strIdx_e7db7ca2d202
category_indexer = strIdx_32a31c399e7b
label_indexer = strIdx_9c2f3315a028


strIdx_9c2f3315a028

convert features to a vector 

In [4]:
import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler().
  setInputCols(Array(
    "Value_index"
    ,"Category_index"
    ,"PHDs"
    ,"paperscore"
    ,"teamsize"
    ,"successes"
    ,"failures"
  )).setOutputCol("assembled")
  


assembler = vecAssembler_d2deec37c04f


vecAssembler_d2deec37c04f

random forest classifier 

In [5]:
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.classification.RandomForestClassificationModel

val rf = new RandomForestClassifier().
  setFeaturesCol("assembled").
  setLabelCol("status").
  setSeed(42)

rf = rfc_b6f3c5e86253


rfc_b6f3c5e86253

create Pipeline 

In [6]:
import org.apache.spark.ml.Pipeline
val pipeline = new Pipeline().setStages(Array(
    value_band_indexer,
    category_indexer,
    label_indexer,
    assembler,
    rf)
  )



pipeline = pipeline_2518e2c5c619


pipeline_2518e2c5c619

create an evaluator  

In [7]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val auc_eval = new BinaryClassificationEvaluator().
  setLabelCol("status").
  setRawPredictionCol("rawPrediction")

auc_eval.getMetricName

val tr = grants.filter("Grant_Application_ID < 6635")
val te = grants.filter("Grant_Application_ID >= 6635")
val training = tr.na.fill(0, Seq("PHDs"))
val test = te.na.fill(0, Seq("PHDs"))

val model = pipeline.fit(training)
val pipeline_results = model.transform(test)
auc_eval.evaluate(pipeline_results)

rf.extractParamMap

auc_eval = binEval_87e17f90ae94
tr = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
te = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
training = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
test = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
model = pipeline_2518e2c5c619
pipeline_results = [Grant_Application_ID: int, Grant_Status: int ... 14 mo...


[Grant_Application_ID: int, Grant_Status: int ... 14 mo...

### Simple Grid Search

Parameter values, different from video>

In [8]:
import org.apache.spark.ml.tuning.ParamGridBuilder

val paramGrid = new ParamGridBuilder().
  addGrid(rf.maxDepth, Array(2, 5)).
  addGrid(rf.numTrees, Array(1, 20)).
  build()

paramGrid = 


Array({
	rfc_b6f3c5e86253-maxDepth: 2,
	rfc_b6f3c5e86253-numTrees: 1
}, {
	rfc_b6f3c5e86253-maxDepth: 2,
	rfc_b6f3c5e86253-numTrees: 20
}, {
	rfc_b6f3c5e86253-maxDepth: 5,
	rfc_b6f3c5e86253-numTrees: 1
}, {
	rfc_b6f3c5e86253-maxDepth: 5,
	rfc_b6f3c5e86253-numTrees: 20
})


### Cross Validation

* Main idea: test with data not used for training
* Split the data several times
* Each time, use part of the data for training and the rest for testing

### k-fold Cross Validation

* Spark supports k-fold cross validation
  - Divides the data into *k* non-overlapping sub-samples
  - Performance is measured by averaging the result of the Evaluator across the *k* folds
  - *k* should be at least 3
  
### Cross Validation

train the model

In [9]:
import org.apache.spark.ml.tuning.CrossValidator

val cv = new CrossValidator().
  setEstimator(pipeline).
  setEvaluator(auc_eval).
  setEstimatorParamMaps(paramGrid).
  setNumFolds(3)
  

cv = cv_c4083266abb9


cv_c4083266abb9

### Final Results

takes a long time 

In [ ]:
val cvModel = cv.fit(training)
val cv_results = cvModel.transform(test)
// with the default parameters we got about 0.908
auc_eval.evaluate(cv_results)

Result: 0.9255259000248613

### Bottom Line

* Nice improvement with little work on our part
  - 0.925 > 0.908
* This did require 12x the computational effort
  - 3x because of the 3-fold cross-validation
  - 4x because of parameter grid search
* But it's embarrassingly parallelizable
  - this is where spark.ml really shines

### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.